## Self-Supervised Pre-training [30 marks]

### What is rotation classification task?
One important self-training task is the task of rotation classification. Here given a set of unlabeled images, we randomly rotate it to either of the following angles {0, 90, 180, 270} degrees and train a image rotation classifier which predicts by what angle the initial image has been rotated to generate the current image. The problem is set up as a classification problem since we only rotate our images by a fixed set of angles as mentioned before and the corresponding ground truth labels being {0, 1, 2, 3}.

<center>
<img src="./fig/Self-training-rot.png" width="524" height="300">
</center>

Once the self-training based pretraining is done, we strip away the final classification layer(which is a linear layer) and add Convolutional or linear layers as per the downstream task's requirement.


### Details of this problem statement.

1. Take the CIFAR-10 dataset, each class has 5000 samples and there are 10 classes.
Split the dataset in 2 parts (A) 40000 and (B) 10000 each with equal number of samples per class in each split.
2. Discard the labels of the samples in the first set.
3. Take a resnet-18 (initialised) and strip the imagenet classification layer with a 4 way classification layer.
4. Train this network on the self-training task of classifiying the rotation of the image.
5. Once this self-supervised pretraining is done, strip the classification layer and add a classification layer for CIFAR-10 classification this is finetuned on the set B for the task of image classification.
6. Log the loss(cross entropy) and accuracies for both the pre-training task and classfication task.

You are free tou use ML API of your choice. Your work will be checked for plagiarism!!

### Citations and helpful references 
[1]. https://www.youtube.com/watch?v=8L10w1KoOU8&t=694s <br>
[2]. https://openaccess.thecvf.com/content_CVPR_2019/papers/Feng_Self-Supervised_Representation_Learning_by_Rotation_Feature_Decoupling_CVPR_2019_paper.pdf

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

In [2]:
# Set the seed for reproducibility
torch.manual_seed(42)

# CIFAR-10 data transformation
transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.ToTensor(),
])

In [ ]:
# Load CIFAR-10 dataset
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Define data loaders
train_loader_A = torch.utils.data.DataLoader(cifar_train, batch_size=64, shuffle=True, num_workers=2)
train_loader_B = torch.utils.data.DataLoader(cifar_test, batch_size=64, shuffle=True, num_workers=2)

# Split dataset into parts A and B
data_A = []
data_B = []
labels_B = []

for data, label in train_loader_A:
    data_A.append(data)

for data, label in train_loader_B:
    data_B.append(data)
    labels_B = labels_B + label.toList()

print(labels_B.shape)
data_A = torch.cat(data_A, dim=0)
data_B = torch.cat(data_B, dim=0)
labels_B = torch.tensor(labels_B)

# Strip labels of the samples in set A
labels_A = torch.zeros(len(data_A), dtype=torch.long)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# ResNet-18 model with a 4-way rotation classification layer
class RotationClassifier(nn.Module):
    def __init__(self):
        super(RotationClassifier, self).__init__()
        resnet = resnet18(pretrained=True)
        # Remove the classification layer
        resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.resnet = nn.Sequential(*resnet)
        self.fc = nn.Linear(resnet.fc.in_features, 4)  # 4-way rotation classification

    def forward(self, x):
        x = self.resnet(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
# Initialize the rotation classifier
rotation_classifier = RotationClassifier()

# Train the rotation classifier on set A
criterion_rotation = nn.CrossEntropyLoss()
optimizer_rotation = optim.SGD(rotation_classifier.parameters(), lr=0.001, momentum=0.9)


In [ ]:
# Training loop for rotation classification task
for epoch in range(5): 
    running_loss = 0.0
    for i, inputs in enumerate(data_A, 0):
        optimizer_rotation.zero_grad()
        outputs = rotation_classifier(inputs)
        loss = criterion_rotation(outputs, labels_A)
        loss.backward()
        optimizer_rotation.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(data_A)}')


In [ ]:
# Strip the classification layer for fine-tuning on CIFAR-10 classification task
classification_model = nn.Sequential(*list(rotation_classifier.resnet.children())[:-1])
fc_layer = nn.Linear(classification_model[-1].in_features, 10)  # 10 classes for CIFAR-10
classification_model.add_module('fc', fc_layer)

# Fine-tune on set B for CIFAR-10 classification
criterion_classification = nn.CrossEntropyLoss()
optimizer_classification = optim.SGD(classification_model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
# Training loop for CIFAR-10 classification task
for epoch in range(5): 
    running_loss = 0.0
    correct = 0
    total = 0
    for i, inputs in enumerate(data_B, 0):
        optimizer_classification.zero_grad()
        outputs = classification_model(inputs)
        loss = criterion_classification(outputs, labels_B)
        loss.backward()
        optimizer_classification.step()
        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels_B.size(0)
        correct += predicted.eq(labels_B).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(data_B)}, Accuracy: {accuracy}%')